In [1]:
%run sim_env_setup.ipynb


In [44]:
# Run this command to disable the motors
p.setJointMotorControl2(scara,1, p.VELOCITY_CONTROL, force=0)

In [52]:
p.setJointMotorControl2(scara, 1, p.TORQUE_CONTROL, force=5)

for _ in range(100):
    p.stepSimulation()
    
p.getJointState(scara, 1)

(1.2953539989079008,
 4.459513452921249,
 (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 1.4052007944850913)